In [20]:
print("hello world!")

hello world!


**What is `faker` in `import faker`**:  
   `faker` is a Python library that generates fake data, such as names, addresses, and emails. It is useful for testing and simulating realistic-looking random data in various domains.

**What is `faker`**:  
   `faker` is a module within the `Faker` library used to generate random fake data, such as names, addresses, and emails. It helps simulate realistic data for development and testing purposes.

**What are `datetime`, `timedelta` in `from datetime import datetime, timedelta`**:  
   `datetime` represents date and time, while `timedelta` represents a duration or difference between two dates. They are used to manipulate and generate dates for order history.

In [21]:
import random
import faker
from datetime import datetime, timedelta
import numpy as np


**What is `random.random()`**:  
   `random.random()` returns a random floating-point number between 0 and 1. It’s used to create a probability condition (like deciding if a product should have variants).

In [22]:
# Constants
PRODUCT_CATEGORIES = ["Clothing", "Groceries", "Electronics"]
PRODUCTS = {
    "Clothing": ["T-shirt", "Jeans", "Jacket"],
    "Groceries": ["Rice", "Pasta", "Olive Oil"],
    "Electronics": ["Laptop", "Smartphone", "Headphones", "ipad"]
}
COLORS = ["Red", "Green", "Blue", "Black", "White"]
VARIANT_CHANCE = 0.5  # 50% chance of a product having variants

# Generate Product Data
def generate_products():
    products = []
    product_id = 1

    for category, product_names in PRODUCTS.items():
        for product in product_names:
            # Check if product should have variants
            if (random.random() < VARIANT_CHANCE) and (category != "Groceries" ):
                # Generate variants
                for color in random.sample(COLORS, random.randint(1, 3)):
                    variant_name = f"{color} {product}"
                    products.append({"product_id": product_id, "category": category, "variant_name": variant_name, "product_name": product})
                    product_id += 1
            else:
                # No variants, add the product itself
                products.append({"product_id": product_id, "category": category, "variant_name": np.nan, "product_name": product})
                product_id += 1
    
    return products


**What is `faker.Faker()`**:  
   `faker.Faker()` initializes a `Faker` instance that provides access to methods for generating fake data. It acts as a generator for creating random names, addresses, emails, etc.


**What are `faker.Faker().name()`, `faker.Faker().email()`, `faker.Faker().address()`, `faker.Faker().phone_number()`**:  
   These methods from `Faker` generate a fake person’s name, email, address, and phone number. They are useful for simulating customer data in an e-commerce scenario.

In [23]:
# Initialize Faker
fake = faker.Faker()

# Generate Customer Data
def generate_customers(num_customers):
    customers = []
    customer_id = 1
    for _ in range(num_customers):
        customers.append({
            "customer_id": customer_id,
            "name": fake.name(),
            "email": fake.email(),
            "address": fake.address(),
            "phone_number": fake.phone_number()
        })

        customer_id += 1
    return customers



**What are `random.choice` and `random.randint` in `generate_orders()`**:  
   `random.choice()` selects a random item from a list (e.g., a random product or customer). `random.randint()` generates a random integer within a specified range (e.g., the number of order items).

**What is `datetime.now() - timedelta(days=random.randint(1, 730))` and `random.randint(1, 5)`**:  
   `datetime.now() - timedelta(days=random.randint(1, 730))` generates a random order date within the last two years. `random.randint(1, 5)` sets a random order quantity between 1 and 5.

  - Since, prive is dynamic and can change with time, I've put price column in orders table

- `uuid4()` in `fake.uuid4()` generates a random unique identifier (UUID) based on random numbers, which is commonly used for creating unique IDs. The datatype is a string representing the UUID in its standard 36-character format (including hyphens).

- The line `order_date.strftime("%Y-%m-%d")` formats the `order_date` into a string that follows the `YYYY-MM-DD` format (e.g., `2023-09-30`).

In [24]:
# Generate Order Data
def generate_orders(products, customers, num_orders=100):
    orders = []
    order_id=1
    for _ in range(num_orders):
        customer = random.choice(customers)
        product = random.choice(products)
        order_date = datetime.now() - timedelta(days=random.randint(1, 730))  # Random date in the last 2 years
        quantity = random.randint(1, 5)
        price = round(random.uniform(10, 500), 2)  # Random price between $10 and $500
        total_price = price * quantity
        
        orders.append({
            "order_id": order_id,
            "customer_id": customer["customer_id"],
            "product_id": product["product_id"],
            "order_date": order_date.strftime("%Y-%m-%d"),
            # The line order_date.strftime("%Y-%m-%d") formats the order_date into a string that follows the YYYY-MM-DD format (e.g., 2023-09-30)
            "quantity": quantity,
            "product_price": price, # Since, prive is dynamic and can change with time, I've put price column in orders table
            "total_price": total_price
        })

        order_id += 1
    
    return orders


In [25]:
# Main Function to Generate All Data
def generate_ecommerce_data():
    # Generate products and customers
    products = generate_products()
    customers = generate_customers(10)  # At least 10 customers

    # Generate orders (At least 2 years of order history)
    orders = generate_orders(products, customers, num_orders=200)  # Generate 200 orders

    return products, customers, orders

# Run the generation
products, customers, orders = generate_ecommerce_data()

# Print the data (For demonstration purposes)
print("\nGenerated Products:\n", products[:5])


Generated Products:
 [{'product_id': 1, 'category': 'Clothing', 'variant_name': 'Red T-shirt', 'product_name': 'T-shirt'}, {'product_id': 2, 'category': 'Clothing', 'variant_name': 'Black T-shirt', 'product_name': 'T-shirt'}, {'product_id': 3, 'category': 'Clothing', 'variant_name': 'White T-shirt', 'product_name': 'T-shirt'}, {'product_id': 4, 'category': 'Clothing', 'variant_name': nan, 'product_name': 'Jeans'}, {'product_id': 5, 'category': 'Clothing', 'variant_name': nan, 'product_name': 'Jacket'}]


- Converting these to dataframe and saving to csv files.

In [26]:
import pandas as pd

df_products = pd.DataFrame(products)
df_customers = pd.DataFrame(customers)
df_orders = pd.DataFrame(orders)

In [27]:
df_products.head()

,product_id,category,variant_name,product_name
0,1,Clothing,Red T-shirt,T-shirt
1,2,Clothing,Black T-shirt,T-shirt
2,3,Clothing,White T-shirt,T-shirt
3,4,Clothing,NaN,Jeans
4,5,Clothing,NaN,Jacket


- To replace all commas in the address column of the df_customers DataFrame with spaces.
- This will help remove complications while reading csv in sql_workbench

In [28]:
df_customers['address'] = df_customers['address'].str.replace(',', ' ', regex=False)
df_customers['address'] = df_customers['address'].str.replace('\n', ' ', regex=False)

In [29]:
df_customers.head()

,customer_id,name,email,address,phone_number
0,1,Adam Harrington,charles75@example.org,83590 Danielle Drive South Brianchester OR 21646,001-525-541-6861x717
1,2,David Lopez,frankmatthew@example.com,427 Pamela Dale Suite 080 West Miguelville VI...,+1-946-605-4049x12570
2,3,Gerald Thompson,fhall@example.net,8917 Stephanie Gardens West Alan PA 73911,5432930381
3,4,Travis Prince,robinsonpaul@example.com,99873 James Cape Apt. 705 Richardstad NM 46262,519-679-4522x9423
4,5,Michael Mitchell,steven81@example.net,29281 Nguyen Valleys Robertstad AZ 91084,3615970595


In [30]:
df_customers.isnull().sum()

customer_id     0
name            0
email           0
address         0
phone_number    0
dtype: int64

In [31]:
df_orders.head()

,order_id,customer_id,product_id,order_date,quantity,product_price,total_price
0,1,4,13,2023-09-11,2,483.39,966.78
1,2,5,11,2024-09-16,4,441.68,1766.72
2,3,4,15,2024-04-09,5,43.21,216.05
3,4,4,5,2022-12-16,4,171.53,686.12
4,5,6,7,2023-07-29,5,388.29,1941.45


In [32]:
df_products.to_csv("products.csv", index=False)
df_customers.to_csv("customers.csv", index=False)
df_orders.to_csv("orders.csv", index=False)

In [33]:
df_products.isnull().sum()

product_id      0
category        0
variant_name    6
product_name    0
dtype: int64